# Transfer Options

There are other options for customizing your transfer command:

In [1]:
'''
Run this cell to setup the tutorial
'''
from opentrons import robot, containers, instruments

plate = containers.load('96-flat', 'B1')

tiprack = containers.load('tiprack-200ul', 'A1')
trash = containers.load('point', 'D2')

pipette = instruments.Pipette(
    axis='b',
    max_volume=200,
    tip_racks=[tiprack],   # attach the tip rack
    trash_container=trash) # attach the trash

<hr></hr>

## Always Get a New Tip

Transfer commands will by default use the same one tip for each well, then finally drop it in the trash once finished.

The pipette can optionally get a new tip at the beginning of each aspirate, to help avoid cross contamination.

In [2]:
robot.clear_commands()

pipette.transfer(
    100,
    plate.wells('A1', 'A2', 'A3'),
    plate.wells('B1', 'B2', 'B3'),
    new_tip='always')    # always get a new tip

for c in robot.commands():
    print(c)

Picking up tip from <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well B1>
Drop_tip at <In Place>
Picking up tip from <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well B2>
Drop_tip at <In Place>
Picking up tip from <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A3>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well B3>
Drop_tip at <In Place>


## Never Get a New Tip

For scenarios where you instead are calling **`.pick_up_tip()`** and **`.drop_tip()`** elsewhere in your protocol, the transfer command can ignore picking up or dropping tips.

In [5]:
robot.clear_commands()

pipette.transfer(
    100,
    plate.wells('A1', 'A2', 'A3'),
    plate.wells('B1', 'B2', 'B3'),
    new_tip='never')    # never pick up or drop a tip

for c in robot.commands():
    print(c)

Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well B1>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well B2>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A3>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well B3>


## Trash or Return Tip

By default, the transfer command will drop the pipette's tips in the trash container. However, if you wish to instead return the tip to it's tip rack, you can set `trash=False`.

In [6]:
robot.clear_commands()

pipette.transfer(
    100,
    plate.wells('A1'),
    plate.wells('B1'),
    trash=False)       # do not trash tip

for c in robot.commands():
    print(c)

Picking up tip from <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well B1>
Returning tip
Drop_tip at <Deck>/<Slot A1>/<Container tiprack-200ul>/<Well E1>


## Touch Tip

A touch-tip can be performed after every aspirate and dispense by setting `touch_tip=True`.

In [11]:
robot.clear_commands()

pipette.transfer(
    100,
    plate.wells('A1'),
    plate.wells('A2'),
    touch_tip=True)     # touch tip to each well's edge

for c in robot.commands():
    print(c)

Picking up tip from <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Touching tip
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Touching tip
Drop_tip at <In Place>


## Blow Out

A blow-out can be performed after every dispense that leaves the tip empty by setting `blow_out=True`.

In [12]:
robot.clear_commands()

pipette.transfer(
    100,
    plate.wells('A1'),
    plate.wells('A2'),
    blow_out=True)      # blow out droplets when tip is empty

for c in robot.commands():
    print(c)

Picking up tip from <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Blow_out at <In Place>
Drop_tip at <In Place>


## Mix Before/After

A mix can be performed before every aspirate by setting `mix_before=()`. The value of `mix_before=` must be a tuple, the *1st* value is the number of repetitions, the *2nd* value is the amount of liquid to mix.

In [13]:
robot.clear_commands()

pipette.transfer(
    100,
    plate.wells('A1'),
    plate.wells('A2'),
    mix_before=(2, 50), # mix 2 times with 50uL before aspirating
    mix_after=(3, 75))  # mix 3 times with 75uL after dispensing

for c in robot.commands():
    print(c)

Picking up tip from <In Place>
Mixing 2 times with a volume of 50ul
Aspirating 50uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Dispensing 50uL at <In Place>
Aspirating 50uL at <In Place>
Dispensing 50uL at <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Mixing 3 times with a volume of 75ul
Aspirating 75uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Dispensing 75uL at <In Place>
Aspirating 75uL at <In Place>
Dispensing 75uL at <In Place>
Aspirating 75uL at <In Place>
Dispensing 75uL at <In Place>
Drop_tip at <In Place>


## Air Gap

An air gap can be performed after every aspirate by setting `air_gap=int`, where the value is the volume of air in microliters to aspirate after aspirating the liquid.

In [14]:
robot.clear_commands()

pipette.transfer(
    100,
    plate.wells('A1'),
    plate.wells('A2'),
    air_gap=20)         # add 20uL of air after each aspirate

for c in robot.commands():
    print(c)

Picking up tip from <In Place>
Aspirating 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A1>
Air gap
Moving to <Well A1>
Aspirating 20uL at <In Place>
Dispensing 20uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Dispensing 100.0uL at <Deck>/<Slot B1>/<Container 96-flat>/<Well A2>
Drop_tip at <In Place>
